In [ ]:
from pathlib import Path
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')


In [ ]:
# Dados ECDC
# ECDC
!curl -s --output '/tmp/ecdc_vacinas.csv' "https://opendata.ecdc.europa.eu/covid19/vaccine_tracker/csv/data.csv"
data = pd.read_csv('/tmp/ecdc_vacinas.csv') # , parse_dates=['data'], index_col='data', dayfirst=True)
data = data.sort_values(['YearWeekISO', 'ReportingCountry'])
data['date'] = data['YearWeekISO'].apply(lambda x: datetime.datetime.strptime(f"{x}-6", "%Y-W%W-%w"))
data.loc[ data['YearWeekISO'] == '2020-W52', 'date' ] += datetime.timedelta(days=-7)
data.loc[ data['YearWeekISO'] == '2020-W53', 'date' ] += datetime.timedelta(days=-7)
data.set_index('date', inplace=True)
df = data[ data['ReportingCountry'] == 'PT' ].copy()
dft = df[ (df['TargetGroup'] == 'ALL') & (df['Region'] == 'PT') ]


In [ ]:
VACINAS_AZ = []
for k,v in dft[dft.Vaccine == 'AZ'][['FirstDose']].iterrows():
    if True:
        vv = int(v)
        kk = k.strftime("%Y-%m-%d")
        #print(f"k={kk} v={vv}")
        VACINAS_AZ.append( (kk, -vv,) )
        kk = (k + datetime.timedelta(days=84)).strftime("%Y-%m-%d")
        #print(f"k={kk} v={vv}")
        VACINAS_AZ.append( (kk, +vv,) )
    elif False:
        for d in range(7, 0, -1):
            vv = float(round(v/7,1))
            kk = (k + datetime.timedelta(days=0 - d)).strftime("%Y-%m-%d")
            #print(f"k={kk} v={vv}")
            VACINAS_AZ.append( (kk, -vv,) )
            kk = (k + datetime.timedelta(days=84 - d)).strftime("%Y-%m-%d")
            #print(f"k={kk} v={vv}")
            VACINAS_AZ.append( (kk, +vv,) )
#VACINAS_AZ

In [ ]:
# Lê dados diários e vacinas
data = pd.read_csv(Path.cwd() / '..' / '..' /'data.csv')
data_vacinas = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas.csv')
data_vacinas_detalhes = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas_detalhe.csv')


In [ ]:
data_vacinas.tail(3)


In [ ]:
def f(valor, plus=False):
    if valor is None: return None
    valor = valor if type(valor) == int else float(valor)
    r = format(valor, ",").replace(".","!").replace(",",".").replace("!",",")
    return f"+{r}" if plus and valor > 0 else r


In [ ]:
def foo(x):
    t = datetime.datetime.strptime(x, '%d-%m-%Y')
    if t >= datetime.datetime.strptime('2021-03-01', '%Y-%m-%d'):
        return t + datetime.timedelta(days=28)
    return t + datetime.timedelta(days=22)


In [ ]:
df = data_vacinas.copy()
df['day'] = df['data'].apply(foo)
df.set_index('day', inplace=True)
df['doses1_7'] = df.doses1.diff(7).div(7)
df['doses2_7'] = df.doses2.diff(7).div(7)
df = df[ df.index >= '2021-01-17' ]
df[['doses1_novas', 'doses2_novas', 'doses1_7', 'doses2_7']].plot(figsize=(20,4))
plt.xticks(list(reversed(list(reversed(list(df.index)))[0::7])), rotation=90)
plt.show()


In [ ]:
# Percentagem vacinados daqui a 21/28 dias quando doses2 seguir a curva de doses1
df = data_vacinas.copy()[['data', 'doses1']]
df['day'] = df['data'].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y'))
df.set_index('day', inplace=True)
for i in VACINAS_AZ:
    df.loc[df.index >= i[0], 'doses1'] += i[1]


doses = 486149.0 # df[df.index == '2021-02-28'].doses1[-1]
df['day'] = df['data'].apply(foo)
for i in range(1,7):
    row = pd.DataFrame([[ '', doses, datetime.datetime.strptime('2021-03-22', '%Y-%m-%d') + datetime.timedelta(days=i) ]], columns=['data', 'doses1', 'day'])
    df = df.append(row)
df = df.sort_values('day')
df.set_index('day', inplace=True)

df['previsao'] = df.doses1
df['previsao_novas'] = df.previsao.diff(1)
df['previsao_7'] = df.previsao.diff(7).div(7)
df_previsao = df


df = data_vacinas.copy()
df['day'] = df['data'].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y'))
df.set_index('day', inplace=True)

df['doses2_7'] = df.doses2.diff(7).div(7)
df = df[ df.index >= '2021-01-17' ]
df_doses2 = df

# --- 

fig, axes = plt.subplots(figsize=(15,5))

axes.plot(df_previsao.previsao_7, label="Previsão (média 7d)", lw=3) #, color="#3a3")
axes.plot(df_doses2.doses2_7, label="Vacinados (média 7d)", lw=4) #, color="orange")

axes.plot(df_previsao.previsao_novas, label="Previsão", marker='s', alpha=0.5, markersize=4) #, ls='--', color="#3a3")
axes.plot(df_doses2.doses2_novas, label="Vacinados", marker='s', alpha=0.5, markersize=4) #, ls='--', color="orange")

axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today())

dia = data_vacinas[-1:]['data'].values[0]
axes.set_title(f"Previsão Vacinados por dia ({dia})")
axes.legend()
plt.ylim(0) # ignore negative dailies
plt.xticks(list(reversed(list(reversed(list(df_previsao.index)))[0::7])), rotation=90)
plt.show()


In [ ]:
df_previsao['previsao_sum'] = df_previsao['previsao_novas'].cumsum()
df_doses2['doses2_sum'] = df_doses2['doses2_novas'].cumsum()


In [ ]:
df = df_doses2.copy()
df.drop(['doses1', 'doses1_novas'], axis=1, inplace=True)
df2 = df_previsao.copy()
#df2.drop(['doses1', 'doses1_novas'], axis=1, inplace=True)

df3 = df2.merge(right=df, how='outer', left_index=True, right_index=True)
df3 = df3[ df3['previsao'] > 0 ]

df3.tail(30).head(4)


In [ ]:
fig, axes = plt.subplots(figsize=(15,5))

axes.plot(df3.previsao, label="Previsão", lw=2)
axes.plot(df3.doses2, label="Vacinados", lw=4)
axes.legend()

ymax = df3.previsao.max()
#print(ymax)
import math
millions = math.ceil(ymax / 1_000_000.0)
for i in range(1, millions):
    axes.axhline(linewidth=2, ls="--", color='white', alpha=0.75, y=i*1_000_000)

axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today())
dia = data_vacinas[-1:]['data'].values[0]
axes.set_title(f"Previsão Vacinados acumulado ({dia})")
plt.xticks(list(reversed(list(reversed(list(df_previsao.index)))[0::7])), rotation=90)
plt.ylim(0)
plt.show()


In [ ]:
fig, axes = plt.subplots(figsize=(15,5))
axes.plot(df3.previsao_sum, label="Previsão", lw=2)
axes.plot(df3.doses2_sum, label="Vacinados", lw=4)
#axes.plot(df_previsao.previsao_sum, label="Previsão", lw=2)
#axes.plot(df_doses2.doses2_sum, label="Vacinados", lw=4)
axes.legend()

ymax = df3.previsao_sum.max()
#print(ymax)
import math
millions = math.ceil(ymax / 1_000_000.0)
for i in range(1, millions):
    axes.axhline(linewidth=2, ls="--", color='white', alpha=0.75, y=i*1_000_000)

axes.axvline(linewidth=2, ls="--", color='white', alpha=0.75, x=datetime.datetime.today())
dia = data_vacinas[-1:]['data'].values[0]
axes.set_title(f"Previsão Vacinados acumulado ({dia})")
plt.ylim(0)
plt.show()


In [ ]:
# detalhe

#print(list(data_vacinas_detalhes.tail(1).columns))


In [ ]:
df['vacinas7'] = df['vacinas'].diff(7).div(7).div(10196709 / 100)

df['vacinas7'].plot(figsize=(15,5))
plt.ylim(0)
plt.show()


In [ ]:
df['doses'].diff(1).tail(10)


In [ ]:
#df[-30:]['pessoas_vacinadas_completamente'].plot()

df[-30:]['doses'].plot()

